In [1]:
import pandas as pd
import dataframe_image as dfi
import requests
import os
import json
import requests
import warnings
from bs4 import BeautifulSoup
os.makedirs("./Result",exist_ok=True)



# 國內成分證券ETF
https://www.twse.com.tw/zh/products/securities/etf/products/domestic.html

In [2]:


url='https://www.twse.com.tw/rwd/zh/ETF/domestic?response=json&_=1681734086399'

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"
}
req = requests.get(url, headers = headers,verify=False)
ETFdata = req.content.decode('utf8')
ETFdf=pd.DataFrame(json.loads(ETFdata)['data'])
ETFdf=ETFdf.dropna()
ETFdf.columns = ['證券代號', '證券簡稱']

C:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.twse.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(



# ETF   持股 Data

In [3]:
# https://www.moneydj.com/ETF/X/Basic/Basic0007A.xdjhtm?etfid=0050.TW
# https://www.moneydj.com/etf/eb/et305001list.djhtm?order=4&c=47
def get_erfBaseData(etfid,rtfdf):
    etfurl='https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid='+etfid+'.TW'
    etfresp = requests.get(etfurl)
    etfresp.encoding = 'utf-8'    #轉換編碼至UTF-8
    print("utl="+etfurl)
    if etfresp.status_code==200 :
        etfsoup = BeautifulSoup(etfresp.text, 'lxml')
        
        #印出表格中的所有文字內容
        for tr in etfsoup.find('table', id="Repeater1").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            row.append(etfid)
            if len(row)>0:
                rtfdf = rtfdf.append([row],ignore_index=True)
        for tr in etfsoup.find('table', id="Repeater2").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            if len(row)>0:
                row.append(etfid)
                rtfdf = rtfdf.append([row],ignore_index=True)

        rtfdf=rtfdf.dropna()
       
        return rtfdf

In [4]:
rtfdf = pd.DataFrame()
for  etfid in ETFdf['證券代號'].tolist():
    rtfdf=get_erfBaseData(etfid,rtfdf)
# 【Output】   
rtfdf.columns = ['股票名稱', '持股(千股)','比例','增減','ETF_Id']
rtfdf.to_csv(r'BaseInfo/ETFBaseData.csv', index = False, encoding='utf-8', errors='ignore')
rtfdf.to_csv(r'Result/ETFBaseDataRead.csv', index = False,encoding='big5', errors='ignore')

utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0050.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0051.TW


C:\Users\user\AppData\Local\Temp\ipykernel_7576\1022840050.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_7576\1022840050.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)


utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0052.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0053.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0055.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0056.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0057.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006203.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006204.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006208.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00690.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00692.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00701.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00713.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00730.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xd

In [5]:



StockBaseData=pd.read_csv('BaseInfo/StockBaseData.csv')
ETFBaseData=pd.read_csv('BaseInfo/ETFBaseData.csv')

In [6]:
# 兩表Join
df=pd.merge(ETFBaseData, StockBaseData, how='left', left_on='股票名稱', right_on='公司簡稱')
# 處理格式問題
df['ETF_Id']='00'+df['ETF_Id'].astype(str)

#處理上市公司產業類別對應不到的狀況
df['上市公司產業類別']=df['上市公司產業類別'].fillna('未分類')
print(df.columns)

Index(['股票名稱', '持股(千股)', '比例', '增減', 'ETF_Id', '公司代號', '公司名稱', '公司簡稱', '產業別',
       '實收資本額', 'Type', '上市公司產業類別', '上櫃公司產業類別', '新編碼'],
      dtype='object')


In [7]:
# 取得 etf 對應持股佔比
df['data']=df.agg(lambda x: f"{x['股票名稱']} ( {x['比例']}%)", axis=1)
df.sort_values('比例', ascending=False)
t=df[['ETF_Id','data']]
t[t['ETF_Id']=='00713']

,ETF_Id,data
1108,00713,統一 ( 9.11%)
1109,00713,可成 ( 6.05%)
1110,00713,台灣大 ( 5.9%)
1111,00713,遠傳 ( 5.39%)
1112,00713,華南金 ( 4.75%)
1113,00713,統一超 ( 3.64%)
1114,00713,亞泥 ( 3.52%)
1115,00713,瑞儀 ( 2.34%)
1116,00713,中鼎 ( 2.23%)
1117,00713,宏全 ( 2.14%)


In [8]:
keydata=df.groupby(['ETF_Id','上市公司產業類別'])['data'].apply(','.join).reset_index()
keydata

,ETF_Id,上市公司產業類別,data
0,0050,光電業,大立光 ( 0.65%)
1,0050,其他,中租-KY ( 0.28%)
2,0050,其他電子業,鴻海 ( 5.65%)
3,0050,半導體業,"台積電 ( 54.15%),聯發科 ( 4.83%),聯電 ( 1.52%),日月光投控 (..."
4,0050,塑膠工業,"南亞 ( 0.6%),台塑 ( 0.5%),台化 ( 0.4%)"
...,...,...,...
711,00946,電子零組件業,"致伸 ( 2.43%),精成科 ( 1.97%),信錦 ( 1.73%),凡甲 ( 0.57..."
712,00946,電腦及週邊設備業,"新普 ( 3.76%),達方 ( 1.04%),和碩 ( 0.53%),威強電 ( 0.35%)"
713,00947,半導體業,"聯發科 ( 11.41%),聯詠 ( 8.06%),瑞昱 ( 7.93%),信驊 ( 5.8..."
714,00947,未分類,"nan ( 1.39%),nan ( 1.39%),nan ( 0.1%),nan ( 0.1%)"


In [9]:
# 取得產業比例排名 by ETF_Id 上市公司產業類別

# 挑選要用的欄位
colList=['ETF_Id','股票名稱', '上市公司產業類別','持股(千股)','比例']
df_=df[colList]
# groupby
g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()
# 排序 取前三
#g.sort_values('比例', ascending=False).groupby(['ETF_Id']).head(3).sort_values(['ETF_Id', '比例'], ascending=[True, False])
# 排序(這個對未來比較有彈性)
g['row_numbers'] =g.groupby('ETF_Id')['比例'].rank(ascending=False,method='dense')
g[g['ETF_Id']=='00713']

C:\Users\user\AppData\Local\Temp\ipykernel_7576\2400789445.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()


,ETF_Id,上市公司產業類別,比例,row_numbers
227,00713,光電業,4.61,9.0
228,00713,其他,8.14,5.0
229,00713,其他電子業,8.70,4.0
230,00713,化學工業,1.13,17.0
231,00713,半導體業,7.57,6.0
232,00713,建材營造,0.90,20.0
233,00713,水泥工業,3.52,10.0
234,00713,汽車工業,1.25,16.0
235,00713,紡織纖維,1.41,14.0
236,00713,航運業,1.86,12.0


In [10]:
#轉成橫的方便看
df1=g[g['row_numbers']==1].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df2=g[g['row_numbers']==2].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df3=g[g['row_numbers']==3].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df_result=df1.merge(df2,on='ETF_Id',suffixes=["","_2"]).merge(df3,on='ETF_Id',suffixes=["","_3"])
df_result = df_result.drop(df_result.filter(regex='row_numbers').columns, axis=1)
df_result

,ETF_Id,上市公司產業類別,比例,data,上市公司產業類別_2,比例_2,data_2,上市公司產業類別_3,比例_3,data_3
0,0050,半導體業,63.85,"台積電 ( 54.15%),聯發科 ( 4.83%),聯電 ( 1.52%),日月光投控 (...",金融保險,11.40,"中信金 ( 1.61%),富邦金 ( 1.43%),國泰金 ( 1.27%),兆豐金 ( 1...",電腦及週邊設備業,5.90,"廣達 ( 1.93%),華碩 ( 0.82%),緯創 ( 0.7%),緯穎 ( 0.53%)..."
1,0051,半導體業,13.80,"矽力*-KY ( 1.92%),創意 ( 1.73%),京元電子 ( 1.66%),力成 (...",電子零組件業,13.10,"台光電 ( 1.8%),嘉澤 ( 1.53%),健策 ( 1.51%),健鼎 ( 1.35%...",電腦及週邊設備業,9.88,"技嘉 ( 1.87%),英業達 ( 1.8%),仁寶 ( 1.77%),宏碁 ( 1.69%..."
2,0052,半導體業,76.18,"台積電 ( 62.13%),聯發科 ( 5.55%),聯電 ( 1.75%),日月光投控 (...",電腦及週邊設備業,8.88,"廣達 ( 2.21%),華碩 ( 0.94%),緯創 ( 0.8%),緯穎 ( 0.62%)...",其他電子業,7.41,"鴻海 ( 6.83%),可成 ( 0.37%),鴻準 ( 0.21%)"
3,0053,半導體業,63.19,"台積電 ( 48.92%),聯發科 ( 5.03%),聯電 ( 1.46%),日月光投控 (...",電腦及週邊設備業,9.78,"廣達 ( 2.32%),緯穎 ( 1.0%),華碩 ( 0.7%),緯創 ( 0.6%),奇...",電子零組件業,8.68,"台達電 ( 2.7%),國巨 ( 0.75%),欣興 ( 0.51%),嘉澤 ( 0.42%..."
4,0056,電腦及週邊設備業,22.94,"華碩 ( 2.58%),廣達 ( 2.55%),和碩 ( 2.53%),英業達 ( 2.36...",半導體業,20.65,"聯詠 ( 3.35%),聯電 ( 3.09%),聯發科 ( 3.04%),瑞昱 ( 2.83...",金融保險,11.32,"兆豐金 ( 2.79%),永豐金 ( 2.61%),中信金 ( 2.42%),元大金 ( 2..."
5,0057,半導體業,59.75,"台積電 ( 49.38%),聯發科 ( 4.41%),聯電 ( 1.3%),日月光投控 ( ...",金融保險,10.60,"中信金 ( 1.39%),富邦金 ( 1.29%),國泰金 ( 1.17%),兆豐金 ( 0...",電腦及週邊設備業,6.79,"廣達 ( 1.75%),華碩 ( 0.73%),緯創 ( 0.57%),緯穎 ( 0.53%..."
6,006203,半導體業,59.89,"台積電 ( 49.4%),聯發科 ( 4.46%),聯電 ( 1.3%),日月光投控 ( 1...",金融保險,10.58,"中信金 ( 1.39%),富邦金 ( 1.29%),國泰金 ( 1.17%),兆豐金 ( 0...",電腦及週邊設備業,6.85,"廣達 ( 1.77%),華碩 ( 0.75%),緯創 ( 0.57%),緯穎 ( 0.56%..."
7,006204,半導體業,43.77,"台積電 ( 32.81%),聯發科 ( 3.83%),聯電 ( 1.33%),日月光投控 (...",金融保險,12.32,"富邦金 ( 1.73%),國泰金 ( 1.48%),中信金 ( 1.27%),兆豐金 ( 0...",電腦及週邊設備業,7.81,"廣達 ( 2.06%),緯穎 ( 0.79%),華碩 ( 0.63%),研華 ( 0.54%..."
8,006208,半導體業,63.93,"台積電 ( 54.2%),聯發科 ( 4.83%),聯電 ( 1.53%),日月光投控 ( ...",金融保險,11.41,"中信金 ( 1.61%),富邦金 ( 1.43%),國泰金 ( 1.27%),兆豐金 ( 1...",其他電子業,5.95,鴻海 ( 5.95%)
9,00690,半導體業,59.93,"台積電 ( 30.77%),聯發科 ( 15.27%),聯電 ( 4.71%),聯詠 ( 2...",電子零組件業,12.70,"台達電 ( 5.91%),國巨 ( 2.04%),欣興 ( 1.67%),嘉澤 ( 1.11...",金融保險,6.90,"富邦金 ( 5.39%),上海商銀 ( 1.51%)"


In [13]:

# export png
df_styled = df_result.style.background_gradient()
df_styled = df_result.style.set_properties(subset=['data','data_2','data_3'], **{'width': '320px'})
dfi.export(df_styled, "Result/mytable.png")

In [14]:
df_styled

,ETF_Id,上市公司產業類別,比例,data,上市公司產業類別_2,比例_2,data_2,上市公司產業類別_3,比例_3,data_3
0,0050,半導體業,63.850000,"台積電 ( 54.15%),聯發科 ( 4.83%),聯電 ( 1.52%),日月光投控 ( 1.32%),聯詠 ( 0.81%),瑞昱 ( 0.6%),世芯-KY ( 0.43%),南亞科 ( 0.19%)",金融保險,11.400000,"中信金 ( 1.61%),富邦金 ( 1.43%),國泰金 ( 1.27%),兆豐金 ( 1.08%),玉山金 ( 0.99%),元大金 ( 0.91%),第一金 ( 0.7%),永豐金 ( 0.68%),合庫金 ( 0.64%),華南金 ( 0.62%),開發金 ( 0.55%),台新金 ( 0.52%),上海商銀 ( 0.4%)",電腦及週邊設備業,5.900000,"廣達 ( 1.93%),華碩 ( 0.82%),緯創 ( 0.7%),緯穎 ( 0.53%),光寶科 ( 0.52%),奇鋐 ( 0.51%),和碩 ( 0.5%),研華 ( 0.39%)"
1,0051,半導體業,13.800000,"矽力*-KY ( 1.92%),創意 ( 1.73%),京元電子 ( 1.66%),力成 ( 1.56%),祥碩 ( 1.06%),力積電 ( 1.0%),華邦電 ( 1.0%),智原 ( 0.94%),愛普* ( 0.63%),旺宏 ( 0.59%),威盛 ( 0.46%),采鈺 ( 0.45%),達發 ( 0.4%),新唐 ( 0.26%),台勝科 ( 0.14%)",電子零組件業,13.100000,"台光電 ( 1.8%),嘉澤 ( 1.53%),健策 ( 1.51%),健鼎 ( 1.35%),群光 ( 1.23%),華通 ( 1.12%),臻鼎-KY ( 1.02%),川湖 ( 1.0%),金像電 ( 0.79%),信邦 ( 0.77%),南電 ( 0.5%),華新科 ( 0.48%)",電腦及週邊設備業,9.880000,"技嘉 ( 1.87%),英業達 ( 1.8%),仁寶 ( 1.77%),宏碁 ( 1.69%),微星 ( 1.53%),佳世達 ( 0.66%),神基 ( 0.56%)"
2,0052,半導體業,76.180000,"台積電 ( 62.13%),聯發科 ( 5.55%),聯電 ( 1.75%),日月光投控 ( 1.52%),聯詠 ( 0.93%),瑞昱 ( 0.69%),世芯-KY ( 0.5%),矽力*-KY ( 0.41%),創意 ( 0.37%),京元電子 ( 0.35%),力成 ( 0.34%),祥碩 ( 0.23%),南亞科 ( 0.22%),華邦電 ( 0.21%),力積電 ( 0.21%),智原 ( 0.2%),愛普* ( 0.13%),旺宏 ( 0.12%),采鈺 ( 0.09%),達發 ( 0.08%),威盛 ( 0.06%),新唐 ( 0.06%),台勝科 ( 0.03%)",電腦及週邊設備業,8.880000,"廣達 ( 2.21%),華碩 ( 0.94%),緯創 ( 0.8%),緯穎 ( 0.62%),光寶科 ( 0.59%),奇鋐 ( 0.59%),和碩 ( 0.57%),研華 ( 0.45%),技嘉 ( 0.4%),仁寶 ( 0.38%),英業達 ( 0.38%),宏碁 ( 0.36%),微星 ( 0.33%),佳世達 ( 0.14%),神基 ( 0.12%)",其他電子業,7.410000,"鴻海 ( 6.83%),可成 ( 0.37%),鴻準 ( 0.21%)"
3,0053,半導體業,63.190000,"台積電 ( 48.92%),聯發科 ( 5.03%),聯電 ( 1.46%),日月光投控 ( 1.45%),聯詠 ( 0.7%),創意 ( 0.59%),瑞昱 ( 0.48%),南亞科 ( 0.4%),矽力*-KY ( 0.29%),力成 ( 0.26%),京元電子 ( 0.24%),祥碩 ( 0.21%),華邦電 ( 0.21%),智原 ( 0.16%),台勝科 ( 0.14%),華泰 ( 0.14%),威盛 ( 0.14%),力積電 ( 0.13%),聯鈞 ( 0.12%),愛普* ( 0.11%),致新 ( 0.1%),矽格 ( 0.1%),聯陽 ( 0.1%),新唐 ( 0.1%),景碩 ( 0.09%),創見 ( 0.09%),義隆 ( 0.09%),旺宏 ( 0.09%),同欣電 ( 0.08%),瑞鼎 ( 0.08%),天鈺 ( 0.07%),力智 ( 0.06%),矽創 ( 0.06%),華東 ( 0.06%),超豐 ( 0.06%),凌陽 ( 0.06%),立積 ( 0.06%),京鼎 ( 0.06%),晶豪科 ( 0.06%),強茂 ( 0.06%),台亞 ( 0.05%),順德 ( 0.05%),南茂 ( 0.05%),盛群 ( 0.04%),威鋒電子 ( 0.04%),菱生 ( 0.04%),偉詮電 ( 0.04%),嘉晶 ( 0.04%),敦泰 ( 0.03%),松翰 ( 0.03%),昇陽半導體 ( 0.02%),光罩 ( 0.02%),矽統 ( 0.01%),福懋科 ( 0.01%),界霖 ( 0.01%)",電腦及週邊設備業,9.780000,"廣達 ( 2.32%),緯穎 ( 1.0%),華碩 ( 0.7%),緯創 ( 0.6%),奇鋐 ( 0.57%),研華 ( 0.57%),光寶科 ( 0.54%),和碩 ( 0.52%),技嘉 ( 0.43%),英業達 ( 0.37%),仁寶 ( 0.31%),微星 ( 0.29%),宏碁 ( 0.29%),佳世達 ( 0.16%),神基 ( 0.15%),神達 ( 0.11%),研揚 ( 0.11%),尼得科超眾 ( 0.09%),凌華 ( 0.09%),威強電 ( 0.08%),華擎 ( 0.07%),樺漢 ( 0.07%),飛捷 ( 0.06%),勤誠 ( 0.06%),友通 ( 0.06%),輔信 ( 0.04%),精元 ( 0.04%),振樺電 ( 0.03%),宏正 ( 0.02%),精英 ( 0.01%),藍天 ( 0.01%),達方 ( 0.01%)",電子零組件業,8.680000,"台達電 ( 2.7%),國巨 ( 0.75%),欣興 ( 0.51%),嘉澤 ( 0.42%),台光電 ( 0.31%),川湖 ( 0.3%),臻鼎-KY ( 0.24%),健策 ( 0.23%),群光 ( 0.23%),金像電 ( 0.22%),健鼎 ( 0.22%),南電 ( 0.22%),華通 ( 0.19%),信邦 ( 0.17%),新日興 ( 0.14%),群電 ( 0.13%),精成科 ( 0.12%),華新科 ( 0.11%),建準 ( 0.11%),康舒 ( 0.1%),毅嘉 ( 0.09%),信錦 ( 0.09%),晶技 ( 0.08%),日電貿 ( 0.07%),興勤 ( 0.07%),敬鵬 ( 0.07%),台郡 ( 0.07%),聯茂 ( 0.07%),瀚宇博 ( 0.07%),廣宇 ( 0.06%),全漢 ( 0.06%),立隆電 ( 0.05%),禾伸堂 ( 0.05%),嘉聯益 ( 0.05%),志超 ( 0.04%),嘉基 ( 0.04%),健和興 ( 0.04%),大毅 ( 0.03%),志聖 ( 0.03%),凱美 ( 0.03%),正崴 ( 0.02%),致伸 ( 0.02%),鉅祥 ( 0.02%),飛宏 ( 0.01%),燿華 ( 0.01%),台虹 ( 0.01%),鎰勝 ( 0.01%)"
4,0056,電腦及週邊設備業,22.940000,"華碩 ( 2.58%),廣達 ( 2.55%),和碩 ( 2.53%),英業達 ( 2.36%),仁寶 ( 2.3%),光寶科 ( 2.12%),緯創 ( 2.11%),微星 ( 2.07%),技嘉 ( 1.74%),宏碁 ( 1.44%),神基 ( 1.14%)",半導體業,20.650000,"聯詠 ( 3.35%),聯電 ( 3.09%),聯發科 ( 3.04%),瑞昱 ( 2.83%),日月光投控 ( 2.47%),京元電子 ( 2.38%),力成 ( 2.29%),新唐 ( 0.76%),台勝科 ( 0.44%)",金融保險,11.320000,"兆豐金 ( 2.79%),永豐金 ( 2.61%),中信金 ( 2.42%),元大金 ( 2.4%),京城銀 ( 1.1%)"
5,0057,半導體業,59.750000,"台積電 ( 49.38%),聯發科 ( 4.41%),聯電 ( 1.3%),日月光投控 ( 1.15%),聯詠 ( 0.73%),瑞昱 ( 0.55%),世芯-KY ( 0.39%),力旺 ( 0.35%),矽力*-KY ( 0.31%),創意 ( 0.3%),環球晶 ( 0.29%),世界 ( 0.24%),南亞科 ( 0.18%),華邦電 ( 0.17%)",金融保險,10.600000,"中信金 ( 1.39%),富邦金 ( 1.29%),國泰金 ( 1.17%),兆豐金 ( 0.97%),玉山金 ( 0.84%),元大金 ( 0.68%),第一金 ( 0.64%),合庫金 ( 0.56%),永豐金 ( 0.55%),開發金 ( 0.5%),華南金 ( 0.49%),台新